#### Diffuser generation tour
首先进行controlnet 源码详解和stable diffusion v1.5 的代码处理

In [ ]:
# from diffusers import DiffusionPipeline
# pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", use_safetensors=True)

In [ ]:
# from diffusers import StableDiffusionPipeline

In [ ]:
# pipeline.to("cuda")
# image = pipeline("A woman with 3D animated cartoon style").images[0]
# image

In [ ]:
# instruct pix 2 pix 复现
# 首先生成数据集
import json
import numpy as np
human_written_prompt_path = "/home/fhx/MyProject/multimodal/instructpix2pix/human-written-prompts.jsonl"
with open(human_written_prompt_path) as fp:
    prompts = [json.loads(line) for line in fp]
# get [index, data]
prompts = np.array_split(list(enumerate(prompts)), 1)[0]
prompts

In [ ]:
# for i, _ in unet.named_parameters():
#     print(i)

In [ ]:
# 初始化stable diffusion pipeline -> decompose 
from PIL import Image
import torch
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel,EulerAncestralDiscreteScheduler
from diffusers.schedulers.scheduling_euler_ancestral_discrete import EulerAncestralDiscreteSchedulerOutput
# 初始化三个部分的sd
def initialize_model(checkpoint_name):
    vae = AutoencoderKL.from_pretrained(checkpoint_name, subfolder="vae", use_safetensors=True)
    tokenizer = CLIPTokenizer.from_pretrained(checkpoint_name, subfolder = "tokenizer")
    text_encoder = CLIPTextModel.from_pretrained(checkpoint_name, subfolder = "text_encoder", 
                                                use_safetensors=True)
    unet = UNet2DConditionModel.from_pretrained(checkpoint_name, subfolder = "unet", 
                                                use_safetensors = True)
    # scheduler = pipeline.scheduler
    return vae, tokenizer, text_encoder, unet

def image_process(image):
    img_copy = image.copy()
    for i in range(image.shape[0]):
        image = img_copy[i, ...]
        image = (image / 2 + 0.5).clamp(0, 1).squeeze()
        image = (image.permute(1, 2, 0) * 255).to(torch.uint8).cpu().numpy()
        image = Image.fromarray(image)
    return image

In [ ]:
class EulerAncestralp2pscheduler(EulerAncestralDiscreteScheduler):
    def __init__(
        self,
        num_train_timesteps: int = 1000,
        beta_start: float = 0.0001,
        beta_end: float = 0.02,
        beta_schedule: str = "linear",
        prediction_type: str = "epsilon"
    ):
        super().__init__(num_train_timesteps, beta_start, beta_end, beta_schedule, prediction_type = prediction_type)

    def step(
        self,
        model_output: torch.FloatTensor,
        timestep,
        sample: torch.FloatTensor,
        generator = None,
        return_dict: bool = True,
    ):
        if self.step_index is None:
            self._init_step_index(timestep)

        sigma = self.sigmas[self.step_index]

        # Upcast to avoid precision issues when computing prev_sample
        sample = sample.to(torch.float32)

        # 1. compute predicted original sample (x_0) from sigma-scaled predicted noise
        if self.config.prediction_type == "epsilon":
            pred_original_sample = sample - sigma * model_output
        elif self.config.prediction_type == "v_prediction":
            # * c_out + input * c_skip
            pred_original_sample = model_output * (-sigma / (sigma**2 + 1) ** 0.5) + (sample / (sigma**2 + 1))
        elif self.config.prediction_type == "sample":
            raise NotImplementedError("prediction_type not implemented yet: sample")
        else:
            raise ValueError(
                f"prediction_type given as {self.config.prediction_type} must be one of `epsilon`, or `v_prediction`"
            )

        sigma_from = self.sigmas[self.step_index]
        sigma_to = self.sigmas[self.step_index + 1]
        sigma_up = (sigma_to**2 * (sigma_from**2 - sigma_to**2) / sigma_from**2) ** 0.5
        sigma_down = (sigma_to**2 - sigma_up**2) ** 0.5

        # 2. Convert to an ODE derivative
        derivative = (sample - pred_original_sample) / sigma

        dt = sigma_down - sigma

        prev_sample = sample + derivative * dt

        device = model_output.device
        noise = torch.randn(model_output.shape, dtype=model_output.dtype, device=device, generator=generator)

        prev_sample = prev_sample + noise * sigma_up

        # Cast sample back to model compatible dtype
        prev_sample = prev_sample.to(model_output.dtype)

        # upon completion increase step index by one
        self._step_index += 1

        if not return_dict:
            return (prev_sample,)

        return EulerAncestralDiscreteSchedulerOutput(
            prev_sample=prev_sample, pred_original_sample=pred_original_sample
        )


In [ ]:
scheduler_super = EulerAncestralDiscreteScheduler.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="scheduler")
scheduler_config = scheduler_super.config

In [ ]:
scheduler = EulerAncestralp2pscheduler(scheduler_config["num_train_timesteps"],
                                                  scheduler_config["beta_start"],
                                                  scheduler_config["beta_end"],
                                                  scheduler_config["beta_schedule"],
                                                  scheduler_config["prediction_type"]
                                                  )

In [ ]:
# 对文本提示词进行编码
# initialzie model

# vae, tokenizer, text_encoder, unet = initialize_model("runwayml/stable-diffusion-v1-5")
# device = "cuda"
# vae.to(device)
# text_encoder.to(device)
# unet.to(device)
# # scheduler = EulerAncestralDiscreteScheduler.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="scheduler")
# from tqdm import tqdm
# for i, prompt in tqdm(prompts[:10]):
#     height, width  = 512, 512
#     num_inference_step = 25
#     guidance_scale = 7.5
#     generator = torch.Generator(device = device).manual_seed(0)
#     batch_size = 1
#     # get text input
#     # print(prompt)
#     text_input = tokenizer(
#         ["Generate similar scene in this batch: " + prompt["input"], prompt["output"]], padding = "max_length", max_length = tokenizer.model_max_length, truncation = True,
#         return_tensors = "pt"
#     )
#     # generate unconditional text embeddings for classifier-free guidance
#     max_length = text_input.input_ids.shape[-1]
#     uncond_input = tokenizer([""] * 2 , padding = "max_length", max_length = max_length,
#                              return_tensors = "pt")
#     with torch.no_grad():
#         text_embd = text_encoder(text_input.input_ids.to(device))[0]
#     # print(text_embd.shape)
#         uncond_embeds = text_encoder(uncond_input.input_ids.to(device))[0]
#         text_embeddings = torch.cat([uncond_embeds, text_embd])
#     # create latent diffusion noise
#     latents = torch.randn(
#         (2, 4, height//8, width//8),
#         generator = generator,
#         device = device
#     )
#     # Denoising image
#     # use default scheduler
#     latents = latents * scheduler.init_noise_sigma
#     scheduler.set_timesteps(num_inference_step)
#     for t in tqdm(scheduler.timesteps):
#         latent_model_input = torch.cat([latents] * 2)
#         latent_model_input = scheduler.scale_model_input(latent_model_input, timestep = t)
#         # predict noise residual 
#         with torch.no_grad():
#             noise_pred = unet(latent_model_input, t, encoder_hidden_states = text_embeddings).sample
#             noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
#             noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)
#             latents = scheduler.step(noise_pred, t, latents).prev_sample
#     print(latents.shape)
#     # decode image
#     latents = 1 / 0.18215 * latents
#     with torch.no_grad():
#         image = vae.decode(latents).sample
#     image_before_edit, image_after_edit = image_process(image[0,...]), image_process(image[1,...])
#     break


In [ ]:
# # dataset processing
# import pandas as pd
# import cv2
# import numpy as np
# import os
# path = "/home/fhx/MyProject/multimodal/dataset/train-00000-of-00001-476d66d124561578.parquet"
# train_path = "/home/fhx/MyProject/multimodal/dataset/ip2p/train"
# valid_path = "/home/fhx/MyProject/multimodal/dataset/ip2p/valid"
# ip2p = pd.read_parquet(path)
# # 取10% 作为validation
# valid = ip2p.sample(int(ip2p.shape[0] * 0.1), axis = 0, random_state=0)
# train = ip2p.iloc[~(ip2p.index.isin(valid.index))]
# def preprocess(ip2p, path):
#     before_path = os.path.join(path, "before_edit")
#     after_path = os.path.join(path, "after_edit")
#     prompt_path = os.path.join(path, "prompt")
#     for i, (img1, img2) in enumerate(zip(ip2p["input_image"], ip2p["edited_image"])):
#         before_edit_img = img1["bytes"]
#         after_edit_img = img2["bytes"]
#         # print(before_edit_img)
#         # break
#         before_edit_img_np = cv2.imdecode(np.frombuffer(before_edit_img, dtype=np.uint8), cv2.IMREAD_COLOR)
#         # print(before_edit_img_np)
#         # break
#         after_edit_img_np = cv2.imdecode(np.frombuffer(after_edit_img, dtype=np.uint8), cv2.IMREAD_COLOR)
#         cv2.imwrite(before_path + f"/image_{i}.jpg", before_edit_img_np)
#         cv2.imwrite(after_path + f"/image_{i}.jpg", after_edit_img_np)
#     # process text
#     # prompt_path = "/home/fhx/MyProject/multimodal/dataset/ip2p/prompt/"
#     for i, text in enumerate(ip2p["edit_prompt"]):
#         with open(os.path.join(prompt_path,f'text_{i}.txt'), "w") as f:
#             f.write(text)
#     print("Finish preprocessing.")
# preprocess(train, train_path)
# preprocess(valid, valid_path)

In [ ]:
# # instruct pix2pix pipeline 
# import torch
# from diffusers import StableDiffusionInstructPix2PixPipeline
# pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(
#     "timbrooks/instruct-pix2pix", torch_dtype=torch.float16
# )

In [ ]:
from PIL import Image
import torch
from torch import nn
from tqdm import tqdm
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, DDIMScheduler
from transformers import CLIPModel, CLIPImageProcessor

from torch.utils.data import Dataset, DataLoader
import os
from PIL import Image
import torchvision.transforms as tfs
from torchvision.transforms import functional as FF
import torchvision.transforms as transforms
import random
class IP2P_dataset(Dataset):

    def __init__(self, path, train, size = 320, format = ".jpg", crop = True, tokenizer = None):
        super(IP2P_dataset, self).__init__()
        self.crop_size = size
        self.format = format
        self.train = train
        self.before_edit_dir = os.listdir(os.path.join(path, "before_edit"))
        # self.before_edit_imgs = [os.path.join()]
        self.before_edit_imgs = os.path.join(path, "before_edit")
        self.after_edit_dir = os.path.join(path, "after_edit")
        self.prompt_dir = os.path.join(path, "prompt")
        self.crop = crop
        # print(self.before_edit_dir)
        self.tokenizer = tokenizer
    def __getitem__(self, idx):
        before_edit = Image.open(os.path.join(self.before_edit_imgs, self.before_edit_dir[idx]))
        name = self.before_edit_dir[idx].split("_")[-1].split(".jpg")[0]
        after_name, prompt_name = "image_" + name + self.format, "text_" + name + ".txt"
        after_edit = Image.open(os.path.join(self.after_edit_dir, after_name))
        with open(os.path.join(self.prompt_dir, prompt_name)) as f:
            prompt = f.read()
        if self.crop:
            i,j,h,w=tfs.RandomCrop.get_params(before_edit,output_size=(self.crop_size,self.crop_size))
            before_edit=FF.crop(before_edit,i,j,h,w)
            after_edit=FF.crop(after_edit,i,j,h,w)
        before_edit, after_edit = self.aug_data(before_edit.convert("RGB"), after_edit.convert("RGB"))
        before_edit = tfs.ToTensor()(before_edit)
        after_edit = tfs.ToTensor()(after_edit)
        prompt = self.tokenizer(
            prompt, padding = "max_length", max_length = self.tokenizer.model_max_length, truncation = True,
            return_tensors = "pt"
        )
        un_cond = self.tokenizer([""] , padding = "max_length", max_length = self.tokenizer.model_max_length,
                             return_tensors = "pt")
        return before_edit, after_edit, prompt, un_cond

    def aug_data(self, data, target):
        if self.train:
            rand_hor=random.randint(0,1)
            rand_rot=random.randint(0,3)
            data=tfs.RandomHorizontalFlip(rand_hor)(data)
            target=tfs.RandomHorizontalFlip(rand_hor)(target)
            if rand_rot:
                data = FF.rotate(data, 90 * rand_rot)
                target = FF.rotate(target, 90 * rand_rot)
        return data, target

    def __len__(self):
        return len(self.before_edit_dir)
    

def image_process(img):
    img_ls = []
    for i in range(img.shape[0]):
        image = img[i, ...]
        image = (image / 2 + 0.5).clamp(0, 1).squeeze()
        image = (image.permute(1, 2, 0) * 255).to(torch.uint8).cpu().numpy()
        image = Image.fromarray(image)
        img_ls.append(image)
    return img_ls
# with torch.no_grad():
#     vae, unet, text_encoder, scheduler = pipe.vae, pipe.unet, pipe.text_encoder, pipe.scheduler
#     ds = IP2P_dataset(path, True, tokenizer=tokenizer, size = 512)
#     loader = DataLoader(ds, batch_size=4)
#     device = "cuda"
#     guidance_scale = 7.5
#     image_guidance_scale = 1.5
#     vae.to(device).to(torch.float16)
#     unet.to(device).to(torch.float16)
#     text_encoder.to(device)
#     batch = next(iter(loader))
#     x, gt, prompt, un_cond = batch
#     x, gt = x.to(torch.float16).to(device), gt.to(torch.float16).to(device)
#     bs, c, h, w = x.shape
#     generator = torch.Generator(device = device).manual_seed(0)
#     # text guide
#     text_embedding_hidden = text_encoder(prompt["input_ids"].to(device))[0]
#     # uncond text guide
#     null_conditioning = text_encoder(un_cond["input_ids"].to(device))[0]  
#     # final text embd
#     prompt_embd = torch.cat([text_embedding_hidden, null_conditioning, null_conditioning])

#     # image final guide
#     # source guide
#     source_imge_embd = vae.encode(x).latent_dist.mode()
#     vae_h, vae_w = source_imge_embd.shape[-2], source_imge_embd.shape[-1]
#     un_cond_image =  torch.zeros_like(source_imge_embd)
#     final_image_embd = torch.cat([source_imge_embd, un_cond_image, un_cond_image])

#     # got latent representation
#     latents = torch.randn(
#     (bs, 4, vae_h, vae_w),
#         generator = generator,
#         device = device
#     ).to(torch.float16)
#     latents = latents * scheduler.init_noise_sigma
#     scheduler.set_timesteps(4)
#     for t in tqdm(scheduler.timesteps):
#         # avoid doing three forward step
#         # do classifier free guidance
#         latent_model_input = torch.cat([latents] * 3)
#         latent_model_input = scheduler.scale_model_input(latent_model_input, timestep = t)
#         # concat latent and images latents
#         latent_model_input = torch.cat([latent_model_input, final_image_embd], dim = 1)

#         noise_pred = unet(latent_model_input, t, encoder_hidden_states = prompt_embd).sample
#         noise_pred_text, noise_pred_image, noise_pred_uncond = noise_pred.chunk(3)
#         noise_pred = (
#             noise_pred_uncond 
#             + guidance_scale * (noise_pred_text - noise_pred_image)
#             + image_guidance_scale * (noise_pred_image - noise_pred_uncond)
#         )
#         latents = scheduler.step(noise_pred, t, latents).prev_sample
#     print(latents.shape)
#     # decode image
#     latents = 1 / 0.18215 * latents
#     edited_image = vae.decode(latents).sample
#     edited_image = image_process(edited_image)

In [ ]:
# lightning training code
from typing import Any
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint,LearningRateMonitor
from pytorch_lightning.strategies import DDPStrategy
from pytorch_lightning.loggers import TensorBoardLogger
import lpips
from utils.EMA import EMA as EMA_callback
from utils.optimizer import get_cosine_schedule_with_warmup
from utils.imgqual_utils import PSNR, SSIM
from utils.loss import L1_Charbonnier_loss
from transformers import CLIPModel, CLIPImageProcessor
from torchmetrics.image.fid import FrechetInceptionDistance

class Instruct_pix2pix(pl.LightningModule):

    def __init__(self, params):
        super(Instruct_pix2pix, self).__init__()
        self.params = params
        self.max_steps = self.params.Trainer.max_steps
        self.save_path = "/home/fhx/MyProject/multimodal/instructpix2pix/diffusers/training_ip2p/save_img"
        # self.initlr = self.params.Trainer.initlr
        self.train_datasets = self.params.Trainer.train_datasets
        self.train_batchsize = self.params.Trainer.train_bs
        self.validation_datasets = self.params.Val.val_datasets
        self.val_batchsize = self.params.Val.val_bs
        self.val_crop = True

        self.conditioning_drop_rate = self.params.Model.conditioning_drop_rate
        self.initlr = self.params.Trainer.initlr #initial learning
        self.crop_size = self.params.Trainer.crop_size #random crop size
        self.num_workers = self.params.Trainer.num_workers

        

        self.loss = L1_Charbonnier_loss()
        self.noise_scheduler = DDIMScheduler(**self.params.scheduler)
        self.noise_scheduler.set_timesteps(self.params.Model.num_test_timesteps)
        self.vae, self.tokenizer, self.text_encoder, self.unet = self.initialize_model(self.params.Model.checkpoint)
        print('training num:',self.train_dataloader().__len__())
        print('validation num:',self.val_dataloader().__len__())
        self.automatic_optimization = False
        self.lpips_fn = lpips.LPIPS(net='alex')
        self.mae = nn.L1Loss()
        self.save_hyperparameters()

        # validation model
        # compute clip similarity
        model_ID = "openai/clip-vit-base-patch16"
        self.clipmodel = CLIPModel.from_pretrained(model_ID)
        self.clipprocessor = CLIPImageProcessor.from_pretrained(model_ID)
        self.guidance_scale = self.params.Model.guidance_scale
        self.image_guidance_scale = self.params.Model.image_guidance_scale
        # data, gt, _, _ = next(iter(loader))
        # data, gt = processor(images=data, return_tensors = "pt"), processor(images=gt, return_tensors = "pt")
        # embedding_a, embedding_b = model.get_image_features(data["pixel_values"]), model.get_image_features(gt["pixel_values"])
        # clip metric
        self.model_ID = "openai/clip-vit-base-patch16"
        self.clip_model, self.clip_processor = self.initialize_clip_metrics(self.model_ID)
        # FID metric
        self.fid = FrechetInceptionDistance(feature = 192).to(self.device)
    def image_process(self, img):
        img_ls = []
        for i in range(img.shape[0]):
            image = img[i, ...]
            image = (image / 2 + 0.5).clamp(0, 1).squeeze()
            image = (image.permute(1, 2, 0) * 255).to(torch.uint8).cpu().numpy()
            image = Image.fromarray(image)
            img_ls.append(image)
        return img_ls
    
    def initialize_clip_metrics(self, checkpoint_name):
        model = CLIPModel.from_pretrained(checkpoint_name)
        processor = CLIPImageProcessor.from_pretrained(checkpoint_name)
        # data, gt = processor(images=edited_image, return_tensors = "pt"), processor(images=gt, return_tensors = "pt")
        # embedding_a, embedding_b = model.get_image_features(data["pixel_values"]), model.get_image_features(gt["pixel_values"])
        model.requires_grad_(False)
        return model, processor

    def compute_clip_metrics(self, x, gt):
        x, gt = self.clip_processor(images=x, return_tensors = "pt"), self.clip_processor(images=gt, return_tensors = "pt")
        embedding_x, embedding_gt = self.clip_model.get_image_features(x["pixel_values"]), self.clip_model.get_image_features(gt["pixel_values"])
        cos_sim = torch.nn.functional.cosine_similarity(embedding_x, embedding_gt)
        return torch.mean(cos_sim).item()
    
    def initialize_model(self, checkpoint_name):
        vae = AutoencoderKL.from_pretrained(checkpoint_name, subfolder="vae", use_safetensors=True)
        tokenizer = CLIPTokenizer.from_pretrained(checkpoint_name, subfolder = "tokenizer")
        text_encoder = CLIPTextModel.from_pretrained(checkpoint_name, subfolder = "text_encoder", 
                                                    use_safetensors=True)
        unet = UNet2DConditionModel.from_pretrained(checkpoint_name, subfolder = "unet", 
                                                    use_safetensors = True)
        
        # modified unet
        in_channels, out_channel = 8, unet.conv_in.out_channels
        unet.register_to_config(in_channels = in_channels)
        with torch.no_grad():
            new_conv_in = nn.Conv2d(
                in_channels=in_channels, out_channels=out_channel, kernel_size=unet.conv_in.kernel_size,
                stride=unet.conv_in.stride, padding=unet.conv_in.padding
            )
            # initialize to zero
            new_conv_in.weight.zero_()
            # copy weight for the first 4 channels
            new_conv_in.weight[:, :4, :, :].copy_(unet.conv_in.weight)
            unet.conv_in = new_conv_in
        vae.requires_grad_(False)
        text_encoder.requires_grad_(False)
        return vae, tokenizer, text_encoder, unet
    
    def lpips_score_fn(self,x,gt):
        self.lpips_fn.to(self.device)
        lp_score = self.lpips_fn(
            gt * 2 - 1, x * 2 - 1
        )
        return torch.mean(lp_score).item()
    
    def configure_optimizers(self):
        parameters = [
            {"params": self.unet.parameters()}
        ]
        optimizer = torch.optim.AdamW(parameters, lr=self.initlr,betas=[0.9,0.999], eps = 1e-6, weight_decay=0.01)
        scheduler2 = get_cosine_schedule_with_warmup(optimizer, 200,self.max_steps) #self.max_steps*0.02,
        
        return [optimizer], [scheduler2]
    
    def add_fid(self, pred, gt):
        self.fid.update(pred, real = False)
        self.fid.update(gt, real = True)

    def training_step(self, batch, batch_idx):
        opt = self.optimzers()
        opt.zero_grad()
        x, gt, prompt, un_cond = batch
        bs, c, h, w = x.shape
        # got latent representation
        latents = self.vae.encode(gt).latent_dist.sample()
        latents = latents * self.vae.config.scaling_factor
        # generate noise
        noise = torch.randn_like(latents).to(self.device)
        timesteps = torch.randint(0, self.noise_scheduler.config.num_train_timesteps, (bs,), device=self.device).long()
        noisy_img = self.noise_scheduler.add_noise(latents, noise, timesteps)
        
        # get text embedding 
        text_embedding_hidden = self.text_encoder(prompt["input_ids"])[0]
        # 使用众数 mode as image input guidance
        source_imge_embd = self.vae.encode(x).latent_dist.mode()

        # classifier free guidance introduced in the source paper
        generator = torch.Generator(device = self.device).manual_seed(10)

        random_p = torch.rand(bs, device = self.device, generator = generator)
        # configure prompt mask
        prompt_mask = random_p < 2 * self.conditioning_drop_rate
        prompt_mask = prompt_mask.reshape(bs, 1, 1)
        
        # text conditioning  
        null_conditioning = self.text_encoder(un_cond)[0]
        # if mask null condtion embd else text embed
        text_embedding_hidden = torch.where(prompt_mask, null_conditioning, text_embedding_hidden)

        # Sample mask for original images
        image_mask_dtype = source_imge_embd.dtype
        image_mask = 1 - (
            (random_p >= self.conditioning_drop_rate).to(image_mask_dtype)
             * (random_p < 3 * self.conditioning_drop_rate).to*(image_mask_dtype)
        )
        image_mask = image_mask.reshape(bs, 1, 1, 1)
        # final image conditioning
        source_imge_embd = image_mask * source_imge_embd

        # final guidance input:
        noisy_img = torch.cat([noisy_img, source_imge_embd], dim = 1)
        model_pred = self.unet(noisy_img, timesteps, text_embedding_hidden).sample
        target = noise if self.noise_scheduler.config.prediction_type == "epsilon" else latents
        loss = self.loss(model_pred, target)
        self.manual_backward(loss)
        opt.step()
        sch = self.lr_schedulers()
        sch.step()

        with torch.no_grad():
            self.log("train_loss", loss, prog_bar = True)
        
        return {"loss", loss}

    def validation_step(self, batch, batch_idx):
        x, gt, prompt, un_cond = batch
        bs, c, h, w = x.shape
       
        # text guide
        text_embedding_hidden = self.text_encoder(prompt["input_ids"])[0]
        # uncond text guide
        null_conditioning = self.text_encoder(un_cond["input_ids"])[0]
        # final text embd
        prompt_embd = torch.cat([text_embedding_hidden, null_conditioning, null_conditioning])
        
        # image final guide
        # source guide
        source_imge_embd = self.vae.encode(x).latent_dist.mode()
        un_cond_image =  torch.zeros_like(source_imge_embd)
        final_image_embd = torch.cat([source_imge_embd, un_cond_image, un_cond_image])
        generator = torch.Generator(device = self.device).manual_seed(10)

        # got latent representation
        latents = torch.randn(
        (bs, 4, source_imge_embd.shape[-2], source_imge_embd.shape[-1]),
            generator = generator,
            device = self.device
        )
        latents = latents * self.noise_scheduler.init_noise_sigma
        for t in tqdm(self.noise_scheduler.timesteps):
            # avoid doing three forward step
            # do classifier free guidance
            latent_model_input = torch.cat([latents] * 3)
            latent_model_input = self.noise_scheduler.scale_model_input(latent_model_input, timestep = t)
            # concat latent and images latents
            latent_model_input = torch.cat([latent_model_input, final_image_embd], dim = 1)

            noise_pred = self.unet(latent_model_input, t, encoder_hidden_states = prompt_embd).sample
            noise_pred_text, noise_pred_image, noise_pred_uncond = noise_pred.chunk(3)
            noise_pred = (
                noise_pred_uncond 
                + self.guidance_scale * (noise_pred_text - noise_pred_image)
                + self.image_guidance_scale * (noise_pred_image - noise_pred_uncond)
            )
            latents = self.noise_scheduler.step(noise_pred, t, latents).prev_sample
        print(latents.shape)
        # decode image
        latents = 1 / 0.18215 * latents
        edited_image = self.vae.decode(latents).sample
        edited_image_ls = self.image_process(edited_image)
        cos_sim = self.compute_clip_metrics(edited_image_ls, gt)
        lpips_score = self.lpips_score_fn(edited_image.float(), gt.float())
        self.add_fid((edited_image*255).to(torch.uint8), (gt*255).to(torch.uint8))
        # log metric
        mae = self.mae(edited_image, gt)
        self.log("MAE", mae, sync_dist = True)
        self.log("LPIPS_score", lpips_score, sync_dist = True)
        self.log("Clip image sim", cos_sim, sync_dist = True)
        print("Finish first step validation. ")
        return {"LPIPS_score":lpips_score, "Clip_image_sim":cos_sim, "MAE":mae}
        
    def on_validation_epoch_end(self) -> None:
        # calculate fid
        fid_score = self.fid.compute()
        # clear cache
        self.fid.reset()
        self.log("Epoch FID: ", fid_score, sync_dist = True)
        return 
    
    def train_dataloader(self):
        
        train_set = IP2P_dataset(self.train_datasets,train=True,size=self.crop_size, tokenizer=self.tokenizer)
        train_loader = DataLoader(train_set, batch_size=self.train_batchsize, shuffle=True, num_workers=self.num_workers)

        return train_loader
    
    def val_dataloader(self):
        val_set = IP2P_dataset(self.validation_datasets,train=False,size=200,crop=True, tokenizer=self.tokenizer)
        val_loader = DataLoader(val_set, batch_size=self.val_batchsize, shuffle=False, num_workers=self.num_workers)

        return val_loader


In [ ]:
from easydict import EasyDict
import yaml

ddp = DDPStrategy(process_group_backend="nccl", find_unused_parameters = True)
config_path = r'/home/fhx/MyProject/multimodal/instructpix2pix/diffusers/training_ip2p/option/IP2P_edited.yaml'
with open(config_path, 'r') as f:
    params = yaml.safe_load(f)
config = EasyDict(params)
model = Instruct_pix2pix(config)
checkpoint_callback = ModelCheckpoint(
    monitor='Clip_image_sim',
    filename='epoch{epoch:02d}-Clip_image_sim-{Clip_image_sim:.3f}-lpips_score-{lpips_score:.4f}',
    auto_insert_metric_name=False,   
    every_n_epochs=1,
    save_top_k=3,
    mode = "max",
    save_last=True
    )
ema_ck = EMA_callback(decay = 0.999)
output_dir = '/home/fhx/MyProject/multimodal/instructpix2pix/diffusers/training_ip2p/lgs/NAFNetguidancecat'
logger = TensorBoardLogger(name=config.log_name,save_dir = output_dir )
lr_monitor_callback = LearningRateMonitor(logging_interval='step')
trainer = pl.Trainer(
    check_val_every_n_epoch=config.Trainer.check_val_every_n_epoch,
    max_steps=config.Trainer.max_steps,
    accelerator=config.Trainer.accelerator,
    devices=config.Trainer.devices,
    precision=config.Trainer.precision,
    accumulate_grad_batches = config.Trainer.accumulate_grad_batches,
    logger=logger,
    strategy="ddp_notebook",
    enable_progress_bar=True,
    log_every_n_steps=config.Trainer.log_every_n_steps,
    callbacks = [checkpoint_callback,lr_monitor_callback, ema_ck]
)
trainer.fit(model)